In [1]:
import os
import math
import wandb
import random
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt


from tensorflow.keras.preprocessing.image import load_img

from tensorflow.keras.models import *
from tensorflow.keras.layers import *
from tensorflow.keras.losses import *
from tensorflow.keras.metrics import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.activations import *

from tensorflow.keras.regularizers import *

from tensorflow.keras.callbacks import *
from keras.preprocessing.image import *
from tensorflow.keras.preprocessing import *

/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.15) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "
2023-04-15 13:48:58.193895: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-15 13:48:58.274257: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-04-15 13:48:58.296191: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory:

In [2]:
# Load the CIFAR-10 dataset
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar100.load_data()

# Normalize pixel values between 0 and 1
x_train = x_train / 255.0
x_test = x_test / 255.0

# Convert labels to one-hot encoded vectors
y_train = keras.utils.to_categorical(y_train, num_classes=100)
y_test = keras.utils.to_categorical(y_test, num_classes=100)

In [3]:
batch_size = 128
epochs = 180
n_classes = 100
learning_rate = 0.1

In [4]:
from keras.layers import Activation
from keras import backend as K

# SwiCh Activation Function
def swich(x):
    return tf.where(x>=0, x, x * (tf.sigmoid(x) * (tf.exp(x) + 1)))

# SwiCh Layer
class Swich(Activation):
    def __init__(self, activation=swich, **kwargs):
        super(Swich, self).__init__(activation, **kwargs)
        self.activation = activation
        self.__name__ = 'swich'

In [5]:
class Resnet:
    def __init__(self, size=44, stacks=3, starting_filter=16):
        self.size = size
        self.stacks = stacks
        self.starting_filter = starting_filter
        self.residual_blocks = (size - 2) // 6
        
    def get_model(self, input_shape=(32, 32, 3), n_classes=100):
        n_filters = self.starting_filter

        inputs = Input(shape=input_shape)
        network = self.layer(inputs, n_filters)
        network = self.stack(network, n_filters, True)

        for _ in range(self.stacks - 1):
            n_filters *= 2
            network = self.stack(network, n_filters)

            
            
            
        network = Activation(swish)(network)
        
        
        
        
        network = AveragePooling2D(pool_size=network.shape[1])(network)
        network = Flatten()(network)
        outputs = Dense(n_classes, activation='softmax', 
                        kernel_initializer='he_normal')(network)

        model = Model(inputs=inputs, outputs=outputs)

        return model
    
    def stack(self, inputs, n_filters, first_stack=False):
        stack = inputs

        if first_stack:
            stack = self.identity_block(stack, n_filters)
        else:
            stack = self.convolution_block(stack, n_filters)

        for _ in range(self.residual_blocks - 1):
            stack = self.identity_block(stack, n_filters)

        return stack
    
    def identity_block(self, inputs, n_filters):
        shortcut = inputs

        block = self.layer(inputs, n_filters, normalize_batch=False)
        block = self.layer(block, n_filters, activation=None)

        block = Add()([shortcut, block])

        return block

    def convolution_block(self, inputs, n_filters, strides=2):
        shortcut = inputs

        block = self.layer(inputs, n_filters, strides=strides,
                           normalize_batch=False)
        block = self.layer(block, n_filters, activation=None)

        shortcut = self.layer(shortcut, n_filters,
                              kernel_size=1, strides=strides,
                              activation=None)

        block = Add()([shortcut, block])

        return block
    
    def layer(self, inputs, n_filters, kernel_size=3,
              
              
              
              
              strides=1, activation=swish, normalize_batch=True):
        
        
        
        
    
        convolution = Conv2D(n_filters, kernel_size=kernel_size,
                             strides=strides, padding='same',
                             kernel_initializer="he_normal",
                             kernel_regularizer=l2(1e-4))

        x = convolution(inputs)

        if normalize_batch:
            x = BatchNormalization()(x)

        if activation is not None:
            x = Activation(activation)(x)

        return x
    
def learning_rate_schedule(epoch):
    new_learning_rate = learning_rate

    if epoch <= 50:
        pass
    elif epoch > 50 and epoch <= 100:
        new_learning_rate = learning_rate * 0.1
    else:
        new_learning_rate = learning_rate * 0.01
        
    print('Learning rate:', new_learning_rate)
    
    return new_learning_rate

def get_random_eraser(p=0.5, s_l=0.02, s_h=0.4, r_1=0.3, r_2=1/0.3, v_l=0, v_h=255, pixel_level=False):
    def eraser(input_img):
        if input_img.ndim == 3:
            img_h, img_w, img_c = input_img.shape
        elif input_img.ndim == 2:
            img_h, img_w = input_img.shape

        p_1 = np.random.rand()

        if p_1 > p:
            return input_img

        while True:
            s = np.random.uniform(s_l, s_h) * img_h * img_w
            r = np.random.uniform(r_1, r_2)
            w = int(np.sqrt(s / r))
            h = int(np.sqrt(s * r))
            left = np.random.randint(0, img_w)
            top = np.random.randint(0, img_h)

            if left + w <= img_w and top + h <= img_h:
                break

        if pixel_level:
            if input_img.ndim == 3:
                c = np.random.uniform(v_l, v_h, (h, w, img_c))
            if input_img.ndim == 2:
                c = np.random.uniform(v_l, v_h, (h, w))
        else:
            c = np.random.uniform(v_l, v_h)

        input_img[top:top + h, left:left + w] = c

        return input_img

    return eraser

In [6]:
resnet = Resnet()

model = resnet.get_model()

optimizer = SGD(learning_rate=learning_rate, momentum=0.9)

model.compile(loss='categorical_crossentropy',
              optimizer=optimizer, metrics=['accuracy'])

model.summary()

2023-04-15 13:49:00.108030: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-04-15 13:49:00.110264: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-04-15 13:49:00.110390: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-04-15 13:49:00.110710: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the approp

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 32, 32, 3)]  0           []                               
                                                                                                  
 conv2d (Conv2D)                (None, 32, 32, 16)   448         ['input_1[0][0]']                
                                                                                                  
 batch_normalization (BatchNorm  (None, 32, 32, 16)  64          ['conv2d[0][0]']                 
 alization)                                                                                       
                                                                                                  
 activation (Activation)        (None, 32, 32, 16)   0           ['batch_normalization[0][0]']

In [7]:
from tensorflow.keras import callbacks
class WandbCallback(callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        wandb.log(logs)

In [8]:
wandb.init(project="CIFAR100", entity="hcim", name='ResNet-SwiCh')

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: chan4im (hcim). Use `wandb login --relogin` to force relogin


In [9]:
lr_scheduler = LearningRateScheduler(learning_rate_schedule)
callbacks = [WandbCallback(), lr_scheduler]

from keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(width_shift_range=4,
                             height_shift_range=4,
                             horizontal_flip=True,
                             preprocessing_function=get_random_eraser(p=1, pixel_level=True))
datagen.fit(x_train)

model.fit(datagen.flow(x_train, y_train, batch_size=batch_size),
          validation_data=(x_test, y_test),
          epochs=epochs, workers=4,
          callbacks=callbacks)

Learning rate: 0.1
Epoch 1/180


2023-04-15 13:49:06.791289: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8100
2023-04-15 13:49:07.805718: I tensorflow/stream_executor/cuda/cuda_blas.cc:1614] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


391/391 [==============================] - 27s 58ms/step - loss: 4.9539 - accuracy: 0.0095 - val_loss: 4.9081 - val_accuracy: 0.0100 - lr: 0.1000
Learning rate: 0.1
Epoch 2/180
391/391 [==============================] - 22s 55ms/step - loss: 4.8885 - accuracy: 0.0089 - val_loss: 4.8670 - val_accuracy: 0.0100 - lr: 0.1000
Learning rate: 0.1
Epoch 3/180
391/391 [==============================] - 22s 55ms/step - loss: 4.8484 - accuracy: 0.0099 - val_loss: 4.8285 - val_accuracy: 0.0100 - lr: 0.1000
Learning rate: 0.1
Epoch 4/180
391/391 [==============================] - 22s 55ms/step - loss: 4.8140 - accuracy: 0.0094 - val_loss: 4.7971 - val_accuracy: 0.0100 - lr: 0.1000
Learning rate: 0.1
Epoch 5/180
391/391 [==============================] - 22s 55ms/step - loss: 4.7848 - accuracy: 0.0100 - val_loss: 4.7721 - val_accuracy: 0.0100 - lr: 0.1000
Learning rate: 0.1
Epoch 6/180
391/391 [==============================] - 22s 55ms/step - loss: 4.7595 - accuracy: 0.0098 - val_loss: 4.7514 - val

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



391/391 [==============================] - 22s 55ms/step - loss: 1.4111 - accuracy: 0.7126 - val_loss: 1.6471 - val_accuracy: 0.6614 - lr: 0.0100
Learning rate: 0.010000000000000002
Epoch 65/180
391/391 [==============================] - 22s 55ms/step - loss: 1.4037 - accuracy: 0.7127 - val_loss: 1.7289 - val_accuracy: 0.6466 - lr: 0.0100
Learning rate: 0.010000000000000002
Epoch 66/180
391/391 [==============================] - 22s 55ms/step - loss: 1.3842 - accuracy: 0.7177 - val_loss: 1.6581 - val_accuracy: 0.6615 - lr: 0.0100
Learning rate: 0.010000000000000002
Epoch 67/180
391/391 [==============================] - 22s 55ms/step - loss: 1.3732 - accuracy: 0.7185 - val_loss: 1.9169 - val_accuracy: 0.6021 - lr: 0.0100
Learning rate: 0.010000000000000002
Epoch 68/180
391/391 [==============================] - 22s 55ms/step - loss: 1.3623 - accuracy: 0.7180 - val_loss: 1.6317 - val_accuracy: 0.6669 - lr: 0.0100
Learning rate: 0.010000000000000002
Epoch 69/180
391/391 [================

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



391/391 [==============================] - 22s 55ms/step - loss: 0.8525 - accuracy: 0.8277 - val_loss: 1.5882 - val_accuracy: 0.6793 - lr: 0.0010
Learning rate: 0.001
Epoch 159/180
391/391 [==============================] - 22s 55ms/step - loss: 0.8638 - accuracy: 0.8249 - val_loss: 1.5702 - val_accuracy: 0.6822 - lr: 0.0010
Learning rate: 0.001
Epoch 160/180
391/391 [==============================] - 22s 55ms/step - loss: 0.8593 - accuracy: 0.8260 - val_loss: 1.5749 - val_accuracy: 0.6819 - lr: 0.0010
Learning rate: 0.001
Epoch 161/180
391/391 [==============================] - 22s 55ms/step - loss: 0.8618 - accuracy: 0.8228 - val_loss: 1.5675 - val_accuracy: 0.6811 - lr: 0.0010
Learning rate: 0.001
Epoch 162/180
391/391 [==============================] - 22s 55ms/step - loss: 0.8599 - accuracy: 0.8235 - val_loss: 1.5661 - val_accuracy: 0.6836 - lr: 0.0010
Learning rate: 0.001
Epoch 163/180
391/391 [==============================] - 22s 56ms/step - loss: 0.8550 - accuracy: 0.8263 - va